In [1]:
import pymysql
# 连接数据库
conn = pymysql.connect(host='219.224.169.45', user='lizimeng', password='codegeass', db='investor')
cursor = conn.cursor()
# 查询2014年各用户交易次数
cursor.execute("select count(*) from chengjiao where date_format(tradedate,'%Y')='2014' group by account")
times2014 = cursor.fetchall()
times2014 = [x[0] for x in times2014]
# 查询2015年各用户交易次数
cursor.execute("select count(*) from chengjiao where date_format(tradedate,'%Y')='2015' group by account")
times2015 = cursor.fetchall()
times2015 = [x[0] for x in times2015]
# 查询2016年各用户交易次数
cursor.execute("select count(*) from chengjiao where date_format(tradedate,'%Y')='2016' group by account")
times2016 = cursor.fetchall()
times2016 = [x[0] for x in times2016]
# 断开数据库
cursor.close()
conn.close()

In [ ]:
width = 100
bins2014 = max(times2014) / width + 1
bins2015 = max(times2015) / width + 1
bins2016 = max(times2016) / width + 1
bins = max(bins2014, bins2015, bins2016)
data2014 = [0] * bins
data2015 = [0] * bins
data2016 = [0] * bins
for x in times2014:
    data2014[x/width] += 1
for x in times2015:
    data2015[x/width] += 1
for x in times2016:
    data2016[x/width] += 1

In [ ]:
from pyecharts import Bar
bar = Bar('用户交易频数')
bar._width = 1000
attr = ['%d~%d'%(width*i+1,width*(i+1)) for i in range(bins)]
bar.add('2014年', attr[:50], data2014[:50], is_datazoom_show=True)
bar.add('2015年', attr[:50], data2015[:50], is_datazoom_show=True)
bar.add('2016年', attr[:50], data2016[:50], is_datazoom_show=True)
bar

# 幂律分布的概率密度函数
$\huge f(x) = \frac{(a-1)}{x_{min}} * (\frac{x}{x_{min}})^{-a} , x\geq x_{min}>0 , a>1$

# 参数的极大似然估计
$\huge \hat{a} = 1 + n * (\sum_{i=1}^{n}\ln\frac{x_i}{x_{min}})^{-1}$

In [ ]:
# 使用极大似然估计计算参数
import math

alpha2014 = 0
for x in times2014:
    alpha2014 += math.log(x)
alpha2014 = 1 + len(times2014) / alpha2014

alpha2015 = 0
for x in times2015:
    alpha2015 += math.log(x)
alpha2015 = 1 + len(times2015) / alpha2015

alpha2016 = 0
for x in times2016:
    alpha2016 += math.log(x)
alpha2016 = 1 + len(times2016) / alpha2016

# 卡方检验
$\huge \chi^2 = \sum_{i=1}^{k}\frac{(f_i-np_i)^2}{np_i}$

# n充分大($n\geq50$)时
$\huge \chi^2\sim\chi^2(k-r-1)$

In [ ]:
pro2014 = []
pro2015 = []
pro2016 = []
for i in range(bins):
    pro2014.append((i*width+1)**(1-alpha2014) - (i*width+width+1)**(1-alpha2014))
    pro2015.append((i*width+1)**(1-alpha2015) - (i*width+width+1)**(1-alpha2015))
    pro2016.append((i*width+1)**(1-alpha2016) - (i*width+width+1)**(1-alpha2016))

chi2014 = 0
chi2015 = 0
chi2016 = 0
for i in range(bins):
    chi2014 += (data2014[i]-len(times2014)*pro2014[i])**2 / len(times2014) / pro2014[i]
    chi2015 += (data2015[i]-len(times2015)*pro2015[i])**2 / len(times2015) / pro2015[i]
    chi2016 += (data2016[i]-len(times2016)*pro2016[i])**2 / len(times2016) / pro2016[i]

In [ ]:
print chi2014, chi2015, chi2016

In [ ]:
from scipy.stats import chisquare
print chisquare(data2014, [x*len(times2014) for x in pro2014])
print chisquare(data2015, [x*len(times2015) for x in pro2015])
print chisquare(data2016, [x*len(times2016) for x in pro2016])

In [ ]:
print alpha2014, alpha2015, alpha2016